<div class="alert alert-block alert-success">
<h1>Step 00: Checking the Raw eeg data</h1>
</div>

Author: Hu Chuan-Peng

Affiliation: School of Pyschology, Nanjing Normal Univeristy

Email: hcp4715@hotmail.com

In this notebook, I checked the raw data and compared it to the raw data from E-Prime. This script serves as a sanity check of the experiment and data.


Summary results of this script: a few issues were found (~7 years after data collection).
* (1) The mask were not presented as assumed in the study design (the eprime procedure in my laptop did not present the mask);
* (2) The interval between trials are shorter than expected, because the ITI was presented immediately after the onset of target;
* (3) There is not fixation at the beginning of each trial;
* (4) The experiment was too long, participants are tired at the end late part of the experiment.

Information that were missing from E-Prime:
* onset delay and timing for prime, mask, and ITI

Also, there were many unnecessary information in E-Prime

In [1]:
%matplotlib inline
import os
import os.path as op
import glob
import mne
import numpy as np
import pandas as pd

import re

from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report

In [2]:
root_dir = os.getcwd().split('2015')[0] + '2015' # make sure that the root dir is `moralSelfEEG2015`
print(root_dir)

/media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1
/media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015


## Check Raw data

Let's first check one block of one participants

In [3]:
subj = 'sub6'
cur_raw_dir = op.join(root_dir, "Analysis_Py", "Exp1", 'RawData' , 'eeg' , subj)
cur_file = glob.glob(op.join(cur_raw_dir, 'sub*_1.vhdr'))
print(cur_file)
cur_raw = mne.io.read_raw(cur_file[0])
cur_raw

['/media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub6/sub6_Moral_asso_1.vhdr']
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub6/sub6_Moral_asso_1.vhdr...
Setting channel info structure...


<RawBrainVision | sub6_Moral_asso_1.eeg, 63 x 162150 (324.3 s), ~67 kB, data not loaded>

In [4]:
cur_raw.annotations

<Annotations | 472 segments: New Segment/ (1), Response/R 1 (20), ...>

Notice that the `time` of EEG data in MNE is in ms.

In [5]:
print('The maximum of time is: ', max(cur_raw.times))
cur_df = cur_raw.to_data_frame()
cur_df[['time', 'FP1', 'FZ']].head(10)

The maximum of time is:  324.298


,time,FP1,FZ
0,0,-7023.925781,-4615.820312
1,2,-7024.707031,-4615.039062
2,4,-7012.646484,-4600.537109
3,6,-7028.369141,-4616.162109
4,8,-7051.171875,-4636.181641
5,10,-7051.074219,-4633.251953
6,12,-7050.341797,-4627.294922
7,14,-7050.244141,-4624.365234
8,16,-7028.564453,-4602.197266
9,18,-7027.490234,-4601.367188


Below is the correspondence between trigger, their code in MNE event_id, and their meaning in the experiment. Note that `Stimulus/S 15: 15` is the same as `Response/R 13: 1013`, which is the correct trial.

    'Response/R 13': 1013, correct trial
    'Response/R 14': 1014, erronous trial

    'Response/R  1': 1001, onset of prime
    'Response/R  2': 1002, onset of prime
    'Response/R  3': 1003, onset of prime
    'Response/R  4': 1004, onset of prime
    'Response/R  5': 1005, onset of prime
    'Response/R  6': 1006, onset of prime

    'Stimulus/S  3': 3,  onset of target
    'Stimulus/S  5': 5,  onset of target
    'Stimulus/S  7': 7,  onset of target
    'Stimulus/S  9': 9,  onset of target
    'Stimulus/S 11': 11, onset of target
    'Stimulus/S 13': 13, onset of target

In [6]:
events, events_id=mne.events_from_annotations(cur_raw,event_id='auto')  
df_ev = pd.DataFrame(events)
df_ev.head()

Used Annotations descriptions: ['New Segment/', 'Response/R  1', 'Response/R  2', 'Response/R  3', 'Response/R  4', 'Response/R  5', 'Response/R  6', 'Response/R 13', 'Response/R 14', 'Stimulus/S  3', 'Stimulus/S  5', 'Stimulus/S  7', 'Stimulus/S  9', 'Stimulus/S 11', 'Stimulus/S 13', 'Stimulus/S 15']


,0,1,2
0,0,0,99999
1,4495,0,1002
2,5050,0,5
3,5758,0,15
4,5758,0,1013


In [7]:
events

array([[     0,      0,  99999],
       [  4495,      0,   1002],
       [  5050,      0,      5],
       ...,
       [154725,      0,      7],
       [155284,      0,     15],
       [155284,      0,   1013]])

In [8]:
events_id

{'New Segment/': 99999,
 'Response/R  1': 1001,
 'Response/R  2': 1002,
 'Response/R  3': 1003,
 'Response/R  4': 1004,
 'Response/R  5': 1005,
 'Response/R  6': 1006,
 'Response/R 13': 1013,
 'Response/R 14': 1014,
 'Stimulus/S  3': 3,
 'Stimulus/S  5': 5,
 'Stimulus/S  7': 7,
 'Stimulus/S  9': 9,
 'Stimulus/S 11': 11,
 'Stimulus/S 13': 13,
 'Stimulus/S 15': 15}

In [26]:
tmp = cur_raw.annotations
type(tmp)

mne.annotations.Annotations

Here we rename the events and change the event id to more meaningful strings

In [9]:
df_ev.rename(columns = {0:'time', 
                        1: 'duration', 
                        2: 'events_id'}, inplace = True)
# df_ev.shape
df_ev['Real_Events'] = None 
df_ev['Code_eprm']   = None
df_ev['Real_Events'].loc[df_ev['events_id'] == 1001] = 'R1_prime_bad_mismatch' 
df_ev['Code_eprm'].loc[df_ev['events_id'] == 1001]   = 16
df_ev['Real_Events'].loc[df_ev['events_id'] == 1002] = 'R2_prime_bad_match'
df_ev['Code_eprm'].loc[df_ev['events_id'] == 1002]   = 32

df_ev['Real_Events'].loc[df_ev['events_id'] == 1003] = 'R3_prime_good_mismatch' 
df_ev['Code_eprm'].loc[df_ev['events_id'] == 1003]   = 48
df_ev['Real_Events'].loc[df_ev['events_id'] == 1004] = 'R4_prime_good_match' 
df_ev['Code_eprm'].loc[df_ev['events_id'] == 1004]   = 64
df_ev['Real_Events'].loc[df_ev['events_id'] == 1005] = 'R5_prime_neut_mismatch' 
df_ev['Code_eprm'].loc[df_ev['events_id'] == 1005]   = 80
df_ev['Real_Events'].loc[df_ev['events_id'] == 1006] = 'R6_prime_neut_match' 
df_ev['Code_eprm'].loc[df_ev['events_id'] == 1006]   = 96

df_ev['Real_Events'].loc[df_ev['events_id'] == 1013] = 'R13' 
df_ev['Real_Events'].loc[df_ev['events_id'] == 1014] = 'R14' 

df_ev['Code_eprm'].loc[df_ev['events_id'] == 1013] = 'Correct_trial' 
df_ev['Code_eprm'].loc[df_ev['events_id'] == 1014] = 'Wrong_trial' 

df_ev['Real_Events'].loc[df_ev['events_id'] == 3] = 'S3_target_bad_mismatch' 
df_ev['Code_eprm'].loc[df_ev['events_id'] == 3] = 3
df_ev['Real_Events'].loc[df_ev['events_id'] == 5] = 'S5_target_bad_match' 
df_ev['Code_eprm'].loc[df_ev['events_id'] == 5] = 5
df_ev['Real_Events'].loc[df_ev['events_id'] == 7] = 'S7_target_good_mismatch' 
df_ev['Code_eprm'].loc[df_ev['events_id'] == 7] = 7
df_ev['Real_Events'].loc[df_ev['events_id'] == 9] = 'S9_target_good_match' 
df_ev['Code_eprm'].loc[df_ev['events_id'] == 9] = 9
df_ev['Real_Events'].loc[df_ev['events_id'] == 11] = 'S11_target_neut_mismatch' 
df_ev['Code_eprm'].loc[df_ev['events_id'] == 11] = 11
df_ev['Real_Events'].loc[df_ev['events_id'] == 13] = 'S13_target_neut_match' 
df_ev['Code_eprm'].loc[df_ev['events_id'] == 13] = 13

df_ev = df_ev[df_ev['events_id'] != 15].copy()     # remove duplicated trigger
df_ev = df_ev[df_ev['events_id'] != 99999].copy()  # remove session marker
print(df_ev.shape)

df_ev['time'] = df_ev['time']*2
df_ev['time_gap'] = df_ev['time'].diff()
df_ev_tar = df_ev.loc[df_ev['events_id'].isin([1013, 1014])].copy()

(360, 5)


/home/hcp4715/miniconda3/envs/mne/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Here we read the csv file from E-Prime

In [10]:
subj = 'sub6'
cur_raw_dir = op.join(root_dir, "Analysis_Py", "Exp1", 'RawData' , 'behav')
cur_file = glob.glob(op.join(cur_raw_dir, 'sub*.csv'))
# print(cur_file)
cur_csv = pd.read_csv(cur_file[0])

cur_csv = cur_csv[cur_csv['BlockList.Sample'].notna()]
cur_csv = cur_csv[cur_csv['BlockList.Sample'] == 1]
# cur_csv.head(9)
cur_csv.columns

Index(['Subject', 'SessionDate', 'SessionTime', 'Block', 'FixDur', 'RespLimit',
       'BlockList.Sample', 'Code', 'Code1', 'Label', 'scramble', 'labelpic',
       'Shape', 'shapepic', 'YesNoResp', 'primeDur', 'maskDur', 'TargetDur',
       'ITIDur', 'Target.OnsetDelay', 'Target.RESP', 'Target.CRESP',
       'Target.ACC', 'Target.OnsetTime', 'Target.RTTime', 'Target.RT',
       'FeedDur', 'TrialList', 'TrialList.Cycle', 'TrialList.Sample'],
      dtype='object')

In [12]:
short_csv = cur_csv[['SessionDate','TrialList.Sample', 'Target.OnsetTime',  'Target.RTTime', 
                     'primeDur', 'maskDur', 'Target.OnsetDelay', 'TargetDur', 'Target.RT', 'ITIDur', 'Target.RESP', 
                     'Code1', 'Code',  
                      'YesNoResp',
                     'Target.CRESP', 'Target.ACC']]
short_csv.head(9)

,SessionDate,TrialList.Sample,Target.OnsetTime,Target.RTTime,primeDur,maskDur,Target.OnsetDelay,TargetDur,Target.RT,ITIDur,Target.RESP,Code1,Code,YesNoResp,Target.CRESP,Target.ACC
48,12-04-2014,1.0,442960.0,443472.0,100,1000,10.0,50,512.0,1357,k,32.0,5.0,Yes,k,1.0
49,12-04-2014,2.0,445550.0,446176.0,100,1000,10.0,50,626.0,1214,m,48.0,7.0,No,m,1.0
50,12-04-2014,3.0,448000.0,448872.0,100,1000,10.0,50,872.0,1340,k,96.0,13.0,Yes,k,1.0
51,12-04-2014,4.0,450570.0,451008.0,100,1000,10.0,50,438.0,1244,m,48.0,7.0,No,m,1.0
52,12-04-2014,5.0,453039.0,453624.0,100,1000,0.0,50,585.0,1383,k,32.0,5.0,Yes,k,1.0
53,12-04-2014,6.0,455649.0,456104.0,100,1000,0.0,50,455.0,1038,m,16.0,3.0,No,m,1.0
54,12-04-2014,7.0,457909.0,458224.0,100,1000,0.0,50,315.0,1218,k,64.0,9.0,Yes,k,1.0
55,12-04-2014,8.0,460349.0,460672.0,100,1000,0.0,50,323.0,1101,m,16.0,3.0,No,m,1.0
56,12-04-2014,9.0,462679.0,463264.0,100,1000,0.0,50,585.0,1282,k,96.0,13.0,Yes,k,1.0


In [13]:
df_ev_tar.shape

(120, 6)

In [14]:
df_ev.head(10)

,time,duration,events_id,Real_Events,Code_eprm,time_gap
1,8990,0,1002,R2_prime_bad_match,32,NaN
2,10100,0,5,S5_target_bad_match,5,1110.0
4,11516,0,1013,R13,Correct_trial,1416.0
5,11580,0,1003,R3_prime_good_mismatch,48,64.0
6,12690,0,7,S7_target_good_mismatch,7,1110.0
8,13964,0,1013,R13,Correct_trial,1274.0
9,14030,0,1006,R6_prime_neut_match,96,66.0
10,15140,0,13,S13_target_neut_match,13,1110.0
12,16540,0,1013,R13,Correct_trial,1400.0
13,16600,0,1003,R3_prime_good_mismatch,48,60.0


The data above revealed two things:
* (1) the intervals between trials, as recorded by trigger in EEG data, was approximately 50 ms, with errors of ~10 ms because of onset delay caused by the blank between trials;
* (2) The trigger between response trigger (R13 or R14) is not at the time of key-pressing. 

Note that in E-prime, the real trialProc is like this:

prime(primeDur, 100 ms) -> blank (maskDur, 1000ms) --> target (dur: 50ms, logging) --> ITI (ITI dur, no logging) -> blank (dur: 50 ms, no logging) --> 

--> Next prime

So, what exactly is the response trigger recorded in EEG data? It is the end of the ITI and before the inter-trial blank screen. We can cross-valid this by taking the differences between target-response interval in the EEG data and the ITI duration in E-prime. As we can see below, it is the duration of target (50 ms) plus the onset delay (~10 ms).

In [11]:
# interval between onset trigger and response trigger, subtract the ITI, equals to the duration of target (50 ms) and ~ 10ms delay
print(1416.0 - 1357.0)
print(1274.0 - 1214.0)
print(1400.0 - 1340.0)
print(1304.0 - 1244.0)
print(1442.0 - 1383.0)
print(1096.0 - 1038.0)

59.0
60.0
60.0
60.0
59.0
58.0


Also, we can confirmed that the time interval between two target onset is consistent between EEG data and the behavioral data as recorded by E-Prime.

In [12]:
# interval between onsets of target are consistent between EEG and E-prime
print("Interval between onsets in EEG: ", 12690 -10100)
print("Interval between onsets in E-prime: ", 445550.0 -442960.0)

Interval between onsets in EEG:  2590
Interval between onsets in E-prime:  2590.0


## Next step: Loop all participants and save the events file

To jointly analyze the EEG data and behavioral data, we need to addd the real reaction times to EEG data.

### First, save the event data of EEG data
Here we will read all the eeg data, recode the events' id, and save them to csv file.

In [15]:
cur_raw_dir = op.join(root_dir, "Analysis_Py", "Exp1", 'RawData' , 'eeg' , "sub*")
cur_raw_dir = glob.glob(cur_raw_dir)
df_list = []
for dir in cur_raw_dir:
    cur_files = glob.glob(op.join(dir, 'sub*.vhdr'))
    for file in cur_files:
        if "29" in file:
            print("skip sub-29 for now")
            continue
        else: 
            cur_raw = mne.io.read_raw(file)
            events, events_id=mne.events_from_annotations(cur_raw, event_id='auto')  
            # events[:10]
            df_ev = pd.DataFrame(events)
            df_ev.rename(columns = {0:'time', 
                                    1: 'duration', 
                                    2: 'events_id'}, inplace = True)
            # df_ev.shape
            df_ev['Real_Events'] = None 
            df_ev['Code_eprm']   = None
            df_ev['Real_Events'].loc[df_ev['events_id'] == 1001] = 'R1_prime_bad_mismatch' 
            df_ev['Code_eprm'].loc[df_ev['events_id'] == 1001]   = 16
            df_ev['Real_Events'].loc[df_ev['events_id'] == 1002] = 'R2_prime_bad_match'
            df_ev['Code_eprm'].loc[df_ev['events_id'] == 1002]   = 32

            df_ev['Real_Events'].loc[df_ev['events_id'] == 1003] = 'R3_prime_good_mismatch' 
            df_ev['Code_eprm'].loc[df_ev['events_id'] == 1003]   = 48
            df_ev['Real_Events'].loc[df_ev['events_id'] == 1004] = 'R4_prime_good_match' 
            df_ev['Code_eprm'].loc[df_ev['events_id'] == 1004]   = 64
            df_ev['Real_Events'].loc[df_ev['events_id'] == 1005] = 'R5_prime_neut_mismatch' 
            df_ev['Code_eprm'].loc[df_ev['events_id'] == 1005]   = 80
            df_ev['Real_Events'].loc[df_ev['events_id'] == 1006] = 'R6_prime_neut_match' 
            df_ev['Code_eprm'].loc[df_ev['events_id'] == 1006]   = 96

            df_ev['Real_Events'].loc[df_ev['events_id'] == 1013] = 'R13' 
            df_ev['Real_Events'].loc[df_ev['events_id'] == 1014] = 'R14' 

            df_ev['Code_eprm'].loc[df_ev['events_id'] == 1013] = 'Correct_trial' 
            df_ev['Code_eprm'].loc[df_ev['events_id'] == 1014] = 'Wrong_trial' 

            df_ev['Real_Events'].loc[df_ev['events_id'] == 3] = 'S3_target_bad_mismatch' 
            df_ev['Code_eprm'].loc[df_ev['events_id'] == 3] = 3
            df_ev['Real_Events'].loc[df_ev['events_id'] == 5] = 'S5_target_bad_match' 
            df_ev['Code_eprm'].loc[df_ev['events_id'] == 5] = 5
            df_ev['Real_Events'].loc[df_ev['events_id'] == 7] = 'S7_target_good_mismatch' 
            df_ev['Code_eprm'].loc[df_ev['events_id'] == 7] = 7
            df_ev['Real_Events'].loc[df_ev['events_id'] == 9] = 'S9_target_good_match' 
            df_ev['Code_eprm'].loc[df_ev['events_id'] == 9] = 9
            df_ev['Real_Events'].loc[df_ev['events_id'] == 11] = 'S11_target_neut_mismatch' 
            df_ev['Code_eprm'].loc[df_ev['events_id'] == 11] = 11
            df_ev['Real_Events'].loc[df_ev['events_id'] == 13] = 'S13_target_neut_match' 
            df_ev['Code_eprm'].loc[df_ev['events_id'] == 13] = 13

            df_ev = df_ev[df_ev['events_id'] != 15].copy()     # remove duplicated trigger
            df_ev = df_ev[df_ev['events_id'] != 99999].copy()  # remove session marker
            print(df_ev.shape)

            df_ev['time'] = df_ev['time']*2
            df_ev['time_gap'] = df_ev['time'].diff()

            # print('Subj # is: ', int(re.findall(r'sub(.*?)\/sub', file)[0]))
            subj_idx = int(re.findall(r'sub(.*?)\/sub', file)[0])
            Block_id = int(re.findall(r'asso_(.*?)\.vhdr', file)[0])
            df_ev['subj_idx'] = subj_idx
            print("subj_idx is: ", subj_idx)
            df_ev['Block_id'] = Block_id
            print("block id is: ", Block_id)

            df_list.append(df_ev)

            
        # df_ev_tar = df_ev.loc[df_ev['events_id'].isin([1013, 1014])].copy()
# cur_raw

Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub10/sub10_Moral_asso_1.vhdr...
Setting channel info structure...
Used Annotations descriptions: ['New Segment/', 'Response/R  1', 'Response/R  2', 'Response/R  3', 'Response/R  4', 'Response/R  5', 'Response/R  6', 'Response/R 13', 'Response/R 14', 'Stimulus/S  3', 'Stimulus/S  5', 'Stimulus/S  7', 'Stimulus/S  9', 'Stimulus/S 11', 'Stimulus/S 13', 'Stimulus/S 15']
(360, 5)
subj_idx is:  10
block id is:  1
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub10/sub10_Moral_asso_2.vhdr...
Setting channel info structure...
Used Annotations descriptions: ['New Segment/', 'Response/R  1', 'Response/R  2', 'Response/R  3', 'Response/R  4', 'Response/R  5', 'Response/R  6', 'Response/R 13', 'Response/R 14', 'Stimulus/S  3', 'Stimulus/S  5', 'Stimulus/S  7', 'Stimulus/S  9', 'Stimulus/S 11', 'Stimulus/S 13', 'Stimulus

/home/hcp4715/miniconda3/envs/mne/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Used Annotations descriptions: ['New Segment/', 'Response/R  1', 'Response/R  2', 'Response/R  3', 'Response/R  4', 'Response/R  5', 'Response/R  6', 'Response/R 13', 'Response/R 14', 'Stimulus/S  3', 'Stimulus/S  5', 'Stimulus/S  7', 'Stimulus/S  9', 'Stimulus/S 11', 'Stimulus/S 13', 'Stimulus/S 15']
(360, 5)
subj_idx is:  10
block id is:  6
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub10/sub10_Moral_asso_7.vhdr...
Setting channel info structure...
Used Annotations descriptions: ['New Segment/', 'Response/R  1', 'Response/R  2', 'Response/R  3', 'Response/R  4', 'Response/R  5', 'Response/R  6', 'Response/R 13', 'Response/R 14', 'Stimulus/S  3', 'Stimulus/S  5', 'Stimulus/S  7', 'Stimulus/S  9', 'Stimulus/S 11', 'Stimulus/S 13', 'Stimulus/S 15']
(360, 5)
subj_idx is:  10
block id is:  7
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub10/sub10_Moral

In [16]:
df = pd.concat(df_list)
df.shape

(78820, 8)

In [17]:
df.to_csv('df_events.csv', index=False)